# Traffic Accidents in Brazillian Federal Highways

In [1]:
from sqlalchemy.exc import OperationalError, DatabaseError
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import mysql.connector as mysql
import pandas as pd
import logging
import os


# INTRO

## Data collection

### Data collection - traffic accidents

### Data collection - GeoJSon

### Data handling
First, we connect to database:

In [4]:
def connect_to_db():
    # Função que SOMENTE conecta na database e retorna seja lá o que for necessário (engine, connection, session...) para operar na database